In [1]:
!pip install torch
!pip install tiktoken


^C
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   - -------------------------------------- 30.7/894.9 kB ? eta -:--:--
   - -------------------------------------- 30.7/894.9 kB ? eta -:--:--
   --- ----------------------------------- 71.7/894.9 kB 653.6 kB/s eta 0:00:02
   ---- ---------------------------------- 92.2/894.9 kB 525.1 kB/s eta 0:00:02
   ----- -------------------------------- 133.1/894.9 kB 605.3 kB/


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import tiktoken
tokenizer=tiktoken.get_encoding('gpt2')
vocab=tokenizer.n_vocab

In [4]:
vocab

50257

In [8]:
data_dir="C:\\Users\\user\\projects\\LLM's-from-scratch\\data.txt"
text = open(data_dir, 'r',encoding='utf-8').read()



In [10]:
import torch
device='cpu'
if torch.cuda.is_available():
    device="cuda"
print(device)
data=torch.tensor(tokenizer.encode(text),dtype=torch.long,device=device)


cuda


In [11]:
len(data)

232724

In [18]:
train_batch_size=16
eval_batch_size=8
context_length=512
train_test_split=0.7

n_len=len(data)
train_data=data[:int(n_len*train_test_split)]
eval_data=data[int(n_len*train_test_split):]

class DataLoader:
    def __init__(self,data,batch_size,context_len):
        self.tokens=data
        self.batch_size=batch_size
        self.context_len=context_len
        self.current_pos=0
    def get_batch(self):
        #create input and target labels "next word perdiction"
        batch_size,context_len=self.batch_size,self.context_len
        startpos=self.current_pos
        endpos=1+batch_size*context_len
        add_tokens=-1
        if endpos>len(self.tokens):
           
            add_tokens=endpos-len(self.tokens)#number of additional tokens required
            endpos=len(self.tokens)
        d=self.tokens[startpos:endpos]
        if add_tokens!=-1:
            d=torch.concat(d,self.tokens[:add_tokens])#adding the missing data from front creating an loop on the data
        input_tokens=(d[:-1]).view(batch_size,context_len)
        target_tokens=d[1:].view(batch_size,context_len)
        self.current_pos+=batch_size*context_len
        if self.current_pos>len(self.tokens):
            self.current_pos=0
        return input_tokens,target_tokens
train_loader=DataLoader(train_data,train_batch_size,context_length)
eval_loader=DataLoader(eval_data,eval_batch_size,context_length)


        




In [19]:
input_tokens,target_tokens=train_loader.get_batch()

In [21]:
print(input_tokens.size(),target_tokens.size())

torch.Size([16, 512]) torch.Size([16, 512])


In [22]:
print(len(eval_data))

69818


In [ ]:
import math
import torch.nn as nn
import torch.nn.functional as F

d_model=512#size of embeddings
n_heads=4#number of heads in multi head attention
n_layers=1#number of attention blocks 

class MultiHeadAttention(nn.Module):
    def __init(self,n_heads:int,d_model:int):
        super.__init__()

        self.d_model=d_model
        self.n_heads=n_heads
        self.head_dim=d_model//n_heads
        assert(self.head_dim*self.heads==self.d_model)
        self.query=nn.linear(d_model,d_model)
        self.key=nn.linear(d_model,d_model)
        self.values=nn.linear(d_model,d_model)
        self.fc_out=nn.linear(d_model,d_model)
        self.drop_out=nn.Dropout(0.2)
    def forward(self,input_tensor:torch.Tensor):
        batch_size,seq_len,d_model=input.size()
        query=self.query(input_tensor).view(batch_size,seq_len,d_model).permute(0,2,1,3)
        key=self.key(input_tensor).view(batch_size,seq_len,d_model).permute(0,2,1,3)
        value=self.value(input_tensor).view(batch_size,seq_len,d_model).permute(0,2,1,3)
        attention_scores=(torch.matmul(query,key.transpose(-2,-1)))/math.sqrt(d_model)
        mask=torch.triu(torch.ones(seq_len,seq_len),diagonal=1).bool().to_device(input_tensor.device)
        attention_scores=attention_scores.masked_fill(mask)
        attention_weights=torch.softmax(attention_scores,dim=-1)
        attention_output=attention_weights.permute(0,2,1,3)
        attnetion_output=attention_output.view(batch_size,seq_len,d_model)
        output=self.fc_out(attnetion_output)
        return output
    
class PositionalEncoder(nn.module):
    def __init__(self,context_length,d_model):
        super.__init__()

        self.context






